
# 목적

> **뉴스 기사 데이터**로부터 시계열(월별) 기반 **핵심 키워드(이슈)** 를 추출하고,    
> **의미적으로 유사한 키워드 그룹화(클러스터링)** 를 통해    
> **특정 기간의 이슈 흐름을 쉽게 파악**할 수 있도록 정제·분석합니다.    

---

## 📂 전체 파이프라인 구성 요약

```bash
/final_preprocessing.json                               ← 기사 원본
├── 1.Event_keyword.ipynb                                ← 월별 키워드 추출 및 클러스터링
├── idf_vectorizer_for_all_corpus.pkl                    ← 전체 코퍼스 기반 TF-IDF 벡터 저장
├── /monthly_results/                                    ← 월별 top30 키워드 저장
├── /monthly_results_cluster/                            ← 월별 키워드 클러스터링 결과
```

---

## 단계별 설명

### ① TF-IDF 기반 월별 핵심 키워드 추출

* **파일**: `1.Event_keyword.ipynb`의 **1번 셀**
* **입력**: `/final_preprocessing.json`
* **출력**: `/monthly_results/YYYY_MM.json` (top30 키워드)

**기능 요약**:

* 기사별 `pubDate` 기준으로 연도·월 분할
* `Okt` 형태소 분석기를 사용하여 명사/동사 기반 토큰화
* 불용어 제거 및 사용자 정의 토크나이저 적용
* `TF-IDF Vectorizer`를 전체 코퍼스에 대해 학습 (`idf_vectorizer_for_all_corpus.pkl`)

  * 이미 벡터라이저가 존재한다면 해당 파일을 재사용
* 월별 TF-IDF 스코어 기준 상위 30개 키워드 추출

**사용된 모델/라이브러리**:

* `scikit-learn`의 `TfidfVectorizer`
* `konlpy`의 `Okt` 형태소 분석기

---

### ② 유사 키워드 클러스터링 (구 단위 의미 통합)

* **파일**: `1.Event_keyword.ipynb`의 **2번 셀**
* **입력**: `/monthly_results/` 내 월별 top30 키워드
* **변경사항**: 노트북 내 `year` 변수를 타겟 연도로 수동 지정 필요
* **출력**: `/monthly_results_cluster/YYYY_MM_cluster.json`

**기능 요약**:

* 추출된 키워드들 중 `같은 의미/주제`를 표현하는 표현들을 묶기 위해 사전 정의된 키워드-구 패턴 매핑 적용
* 예: `"북한군"`, `"조선인민군"` → `"북한 군사"`로 통합
* 이 과정을 통해 시각화 및 인사이트 분석 시 **중복/표현차이로 인한 혼란 방지**

**사용된 모델/기법**:

* 딥러닝 모델은 사용되지 않고, 사전 정의된 단어 패턴 매핑 기반 수동 군집화
* 필요시 `Cosine Similarity`, `FastText`, `Sentence-BERT` 등 확장 가능

---

## 전체 로직 요약 (한 줄씩 요약)

| 단계  | 설명                         | 주요 도구                       |
| --- | -------------------------- | --------------------------- |
| 1단계 | 뉴스 기사 월별 분할 및 텍스트 정제       | `pandas`, `datetime`        |
| 2단계 | Okt 기반 토큰화 + 불용어 제거        | `konlpy`, 사용자 정의            |
| 3단계 | 전체 기사 기반 TF-IDF 학습 (또는 로드) | `TfidfVectorizer`, `joblib` |
| 4단계 | 월별 키워드 상위 30개 추출           | TF-IDF 스코어 기준               |
| 5단계 | 유사 키워드 표현 통합 (의미 기반 군집화)   | 패턴 기반 수동 클러스터링              |

---

## 활용 예시

* 기간별 핵심 이슈 변화 시계열 분석
* 시각화용 월별 키워드 워드클라우드 생성
* 특정 키워드 중심의 상관 이슈 탐색
* LLM 기반 요약 전, 사전 키워드 요약 제공

---

## 주요 산출물 위치

| 디렉터리                                | 내용                          |
| ----------------------------------- | --------------------------- |
| `/monthly_results`                  | 월별 Top 30 TF-IDF 키워드 (JSON) |
| `/monthly_results_cluster`          | 의미 통합된 키워드 그룹 (JSON)        |
| `idf_vectorizer_for_all_corpus.pkl` | 재사용 가능한 전체 코퍼스 기반 벡터라이저     |

